In [1]:
## this notebook assumes that you have numpy, matlplotlib and networkx in your python environment
## if not, run "pip install" or your favorite package manager to install them first

import csv
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline


In [25]:
# download this file http://syntagmatic.github.io/exposedata/marvel/data/hero-network.csv
# CHANGE THE NEXT LINE TO YOUR FILE PATH
marvel_hero_file = './graph-dataset/hero-network.csv'

In [26]:
# borrowing functions and processing from here 
# https://medium.com/@jmandalihan.msds2023/exploring-the-marvel-universe-through-network-analysis-a-starters-guide-88eb40e85c47

# Load data from the CSV file

def load_data_from_csv(file_path):
    data = []
    with open(file_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip the header row
        for row in reader:
            data.append((row[0], row[1]))
    return data


In [29]:

raw_data = load_data_from_csv(marvel_hero_file)

len(raw_data)

574466

In [30]:
# Create an undirected graph
hero_graph = nx.Graph()

# Add edges from the dataset
for edge in raw_data:
    hero_graph.add_edge(edge[0], edge[1])
    
print('total number nodes and edges', hero_graph.number_of_nodes(), hero_graph.number_of_edges() ) 
print ('average number of edges per node', hero_graph.number_of_edges()/hero_graph.number_of_nodes())
print ('density of the Hero social network', nx.density(hero_graph))


total number nodes and edges 6426 167219
average number of edges per node 26.022253345782758
density of the Hero social network 0.00810031232553549


In [31]:
# Create a directed graph
hero_di_graph = nx.DiGraph()

# Add edges from the dataset
for edge in raw_data:
    hero_di_graph.add_edge(edge[0], edge[1])
    
print('total number nodes and edges:', hero_di_graph.number_of_nodes(), hero_di_graph.number_of_edges() ) 
print ('average number of edges per node', hero_di_graph.number_of_edges()/hero_di_graph.number_of_nodes())
print ('density of the Hero social network', nx.density(hero_di_graph))
## why is the number of edges smaller than the number of lines in the CSV?

total number nodes and edges: 6426 224181
average number of edges per node 34.88655462184874
density of the Hero social network 0.005429813948925874


## Question 1

NetworkX inbuild adjacency reader can also read this file `graph_from_adj = nx.read_adjlist(marvel_hero_file)`. 
BUT the outcome is different those from our custom CSV reader. How is it different? What commands/functions from the shell or networkx can you use to expose the differences.

What could have caused this discrepancy? 
Examine the file content, explain the differences, and determine which version you would use? 


## Question 2

Compare how many lines were in the input CSV file, and how many nodes and edges are in either of the networkX graphs. 

Why is the number of edges of the two graphs different?
Why is the number of edges smaller than the number of lines in the CSV?

hint: shell commands such as this may provide a hint `sort graph-dataset/hero-network.csv | uniq | wc -l`

Sort: sorts the lines of files and outout in line
uniq: remove duplicate lines
According to the shell command the unique lines in the csv is 224181 lines,
which is the same as the number of edges in the directed graph 
The undirected graph have less edge may be casuse connections in both way make it differet

In [22]:
# Who are the most connected nodes? does this list surprise you

degree_tuple = sorted(hero_graph.degree(), key=lambda x: x[1], reverse=True)

print(degree_tuple[:15])

[('CAPTAIN AMERICA', 1908), ('SPIDER-MAN/PETER PAR', 1737), ('IRON MAN/TONY STARK ', 1522), ('THING/BENJAMIN J. GR', 1416), ('MR. FANTASTIC/REED R', 1379), ('WOLVERINE/LOGAN ', 1371), ('HUMAN TORCH/JOHNNY S', 1361), ('SCARLET WITCH/WANDA ', 1325), ('THOR/DR. DONALD BLAK', 1289), ('BEAST/HENRY &HANK& P', 1267), ('VISION ', 1241), ('INVISIBLE WOMAN/SUE ', 1236), ('HAWK', 1175), ('WASP/JANET VAN DYNE ', 1091), ('ANT-MAN/DR. HENRY J.', 1082)]


## Question 3

Wait ... my favorite hero is Black Panther / Wong / whoever, 
Write few lines of code that prints the top friends (e.g. top 10 by degree) of any (your favorite) character. 

In [38]:
# put your code here 
# Do it on LITTLE, ABNER
neighbours_of_ABNER = hero_graph.neighbors('LITTLE, ABNER')
degrees = hero_graph.degree(neighbours_of_ABNER)
sorted_neighbours = sorted(degrees, key=lambda x: x[1], reverse=True)
print(sorted_neighbours[:15])

[('IRON MAN/TONY STARK ', 1522), ('THOR/DR. DONALD BLAK', 1289), ('JARVIS, EDWIN ', 986), ("BLACK PANTHER/T'CHAL", 711), ('BINARY/CAROL DANVERS', 537), ('IRON MAN IV/JAMES R.', 465), ('HOGAN, VIRGINIA PEPP', 260), ('JOCASTA', 217), ('FUJIKAWA, RUMIKO', 73), ('MADAME MENACE/SUNSET', 65), ('FIN FANG FOOM/MIDGAR', 62), ('CARNIVORE/COUNT ANDR', 45), ('DECAY II/YOSHIRO HAC', 38), ('TEMPEST II/NICOLETTE', 36), ('INFERNO III/SAMANTHA', 36)]


## Question 4

Who are in the "power center" of the Marvel universe? 
Obtain the mutual connections amongst the top $N_0=20$ characters **by their degree** (computed above). 
* How many edges are there between these characters?
* Are these characters all connected, or do the Marvel network break into several pieces if we only look at the top characters? 
* Do the set of top characters change if we look at the *directed graph* instead?

In [43]:
# put your code here 
degree_tuple = sorted(hero_graph.degree(), key=lambda x: x[1], reverse=True)
top_character  = degree_tuple[:20]
# find edges between these characters
top_connection = 0
for i in range(20):
        for j in range(i+1, 20):
            #print(top_character[i][0], top_character[j][0], hero_graph.has_edge(top_character[i][0], top_character[j][0]))
                top_connection += 1;
print('top_connection', top_connection)
# are they all connected 
# Yes they are there are totaly 190 that is the maximum possible number of edges between 20 nodes
# it is a connected graph (looking only top characters)

top_connection 190


## Question 5 Transitivity and clustering


* Compute the transitivity of the whole marvel hero graph. 
* Compute the clustering coefficients of each node, examine the nodes with the top clustering coefficient, and also the clustering coefficients of the top nodes (by degree) 
* Do the above for the topK subgraph. 
  * Compared to the whole graph, are the transitivity and clustering coefficients larger or smaller? why? 

In [46]:
# put your code here 
# 1 the tarnsitivity of the graph
# 2 the clustering coefficient of each node 
def local_clustreing_coefficient(node,graph):
        if(graph.degree(node) < 2):
                return 0
        neigbours = graph.neighbors(node)
        degree = graph.degree(node)
        connection_between_neighbours = 0
        for i in neigbours:
            for j in neigbours:
                if i != j:
                    connection_between_neighbours += graph.has_edge(i, j)
        return connection_between_neighbours / (degree*(degree-1)/2)        
        # C_i = connection_between_neighbours / (degree*(degree-1)/2)
# run the function for each node 
clustering_coefficient = {}
for node in hero_graph.nodes():
    clustering_coefficient[node] = local_clustreing_coefficient(node,hero_graph)
# Do it for top K subgraph


In [47]:
print(clustering_coefficient)

{'LITTLE, ABNER': 0.03896103896103896, "BLACK PANTHER/T'CHAL": 3.565697985380638e-05, 'PRINCESS ZANDA': 0.10909090909090909, 'STEELE, SIMON/WOLFGA': 0.2222222222222222, 'FORTUNE, DOMINIC': 0.0761904761904762, 'ERWIN, CLYTEMNESTRA': 0.002185792349726776, 'IRON MAN/TONY STARK ': 5.183670401501191e-06, 'IRON MAN IV/JAMES R.': 4.6347793845012975e-05, 'RAVEN, SABBATH II/EL': 0.14545454545454545, 'CARNIVORE/COUNT ANDR': 0.02727272727272727, 'GHOST': 0.05546218487394958, 'ZIMMER, ABE': 0.005477308294209703, 'FU MANCHU': 0.022067363530778164, 'TARR, BLACK JACK': 0.04367816091954023, 'SMITH, SIR DENIS NAY': 0.042328042328042326, 'SHANG-CHI': 0.005124359455068116, 'STARSHINE II/BRANDY ': 0.01117196056955093, 'ROM, SPACEKNIGHT': 0.00529045643153527, 'MAN-THING/THEODORE T': 0.0021617931505290705, 'WU, LEIKO': 0.007399577167019027, 'DOCTOR DREDD': 0.07692307692307693, 'RESTON, CLIVE': 0.04435483870967742, 'JACKSON, STEVE': 0.004024144869215292, 'MYSTIQUE/RAVEN DARKH': 0.003392662083145923, 'BLOB/FR

## Question 6 Weighted network

One possible interpretation for the duplicated edges is that the data is collected over different comic issues / films, and characters appearing together across different publications will have their edges represented multiple times. 

* Now generate a weighted network using `raw_data` read from csv above. 


In [ ]:
# put your code here 

* What is the maximum weight of an edge in the entire Marvel network, between which two heroes? 
* Does the top K=50 network get more interesting using weighted edges? What is the degree distribution like for the topK network?

In [ ]:
# put your code here

## Question 7 Your own network question

What computational question do you want to ask about the Marvel network? 
Try asking one that can be answered by running some algorithm over it. 